In [1]:
import requests
import json
import numpy as np
import pandas as pd
from datetime import datetime
today = datetime.now()

def odds_conv(odds):
    """
    Convert odds to implied probabilities.

    Args:
        odds (float): The odds to convert.

    Returns:
        float: The implied probability.
    """
    if odds > 0:
        return 1 / (odds / 100 + 1)
    else:
        return 1 / (1 + 100 / abs(odds))

def fair_odds(odds_a, odds_b):
    """
    Calculate fair odds for two events.

    Args:
        odds_a (float): The odds for event A.
        odds_b (float): The odds for event B.

    Returns:
        tuple: A tuple containing the fair odds for event A and event B.
    """
    fair_a = odds_conv(odds_a) / (odds_conv(odds_a) + odds_conv(odds_b))
    fair_b = odds_conv(odds_b) / (odds_conv(odds_a) + odds_conv(odds_b))
    return (fair_a, fair_b)

def avg(x, y):
    """
    Calculate the average of two numbers.

    Args:
        x (float): The first number.
        y (float): The second number.

    Returns:
        float: The average of x and y.
    """
    return (x + y) / 2

def profit(stake, odds):
    """
    Calculate profit from a bet.

    Args:
        stake (float): The amount of the stake.
        odds (float): The odds for the bet.

    Returns:
        float: The profit from the bet.
    """
    if odds > 0:
        return odds / 100 * stake
    else:
        return 100 / abs(odds) * stake

def kelly_criterion(b, p, q):
    """
    Calculate the Kelly Criterion for optimal betting.

    Args:
        b (float): The bankroll.
        p (float): The probability of winning.
        q (float): The probability of losing.

    Returns:
        float: The optimal bet size according to the Kelly Criterion.
    """
    return (b * p - q) / b

def width(a, b):
    """
    Calculate the width between two numbers.

    Args:
        a (float): The first number.
        b (float): The second number.

    Returns:
        int: The absolute width between a and b.
    """
    if a < 0 and b < 0:
        return abs(int(str(a)[-2:]) + int(str(b)[-2:]))
    else:
        return abs(int(str(a)[-2:]) - int(str(b)[-2:]))

def get_sport_odds(api_key, sport_key='americanfootball_ncaaf', regions='us'):
    """
    Fetches sports odds data from the Odds API.

    Args:
        api_key (str): API key for authentication.
        sport_key (str): Key identifying the specific sport.
        regions (str): Region for which odds are to be fetched.

    Returns:
        dict: JSON data containing odds information for the specified sport.
    """
    base_url = f'https://api.the-odds-api.com/v4/sports/{sport_key}/odds'
    params = {
        'apiKey': api_key,
        'regions': regions,
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f'Failed to retrieve data. Status code: {response.status_code}')
        return None

def process_odds_data(data):
    """
    Processes the odds data and creates a DataFrame.

    Args:
        data (dict): JSON data containing odds information.

    Returns:
        pd.DataFrame: Processed data in the form of a DataFrame.
    """
    data_list = []
    for game in data:
        game_date = game['commence_time']
        sport = game['sport_title']
        home_team = game['home_team']
        away_team = game['away_team']
        for bookmaker in game['bookmakers']:
            bookmaker_name = bookmaker['title']
            for market in bookmaker['markets']:
                if market['key'] == 'h2h':
                    for outcome in market['outcomes']:
                        team_name = outcome['name']
                        if team_name.lower() == home_team.lower():
                            team_type = 'home'
                        elif team_name.lower() == away_team.lower():
                            team_type = 'away'
                        else:
                            team_type = 'unknown'
                        row = {
                            'date': game_date,
                            'sport': sport,
                            'home_team': home_team,
                            'away_team': away_team,
                            'bookmaker': bookmaker_name,
                            'team_type': team_type,
                            'price': outcome['price']
                        }
                        data_list.append(row)
    return pd.DataFrame(data_list)

def pivot_and_clean_data(df):
    """
    Pivots the DataFrame and fills NaN values with a placeholder.

    Args:
        df (pd.DataFrame): Raw DataFrame containing odds information.

    Returns:
        pd.DataFrame: Pivoted and cleaned DataFrame.
    """
    pivoted_df = df.pivot_table(index=['date', 'sport', 'home_team', 'away_team', 'team_type'],
                                columns='bookmaker',
                                values='price',
                                aggfunc='first')
    pivoted_df.reset_index(inplace=True)
    pivoted_df['team_type'] = pivoted_df.apply(lambda row: row['away_team'] if row['team_type'] == 'away' else row['home_team'], axis=1)
    pivoted_df.rename(columns={'team_type': 'team'}, inplace=True)
    pivoted_df.fillna('N/A', inplace=True)
    return pivoted_df

# Example usage
API_KEY = '287f26ecf5b58ab73781cfaf78310c08'
sport_key = 'americanfootball_ncaaf'
data = get_sport_odds(API_KEY, sport_key)
if data:
    processed_data = process_odds_data(data)
    pivoted_data = pivot_and_clean_data(processed_data)

In [ ]:
def ev(dk, pin, bmgm, stake, bankroll):
    """
    Calculate expected value and optimal bet.

    Args:
        dk (tuple): A tuple containing odds for two events.
        pin (tuple): A tuple containing odds for event A and B from one sportsbook.
        bmgm (tuple): A tuple containing odds for event A and B from another sportsbook.
        stake (float): The stake amount.
        bankroll (float): The available bankroll.

    Returns:
        None
    """
    fair_pin = fair_odds(pin[0], pin[1])
    fair_bmgm = fair_odds(bmgm[0], bmgm[1])
    fair = (avg(fair_pin[0], fair_bmgm[0]), avg(fair_pin[1], fair_bmgm[1]))

    a_profit = profit(stake, dk[0])
    b_profit = profit(stake, dk[1])
    a_ev = a_profit * fair[0] - stake * fair[1]
    b_ev = b_profit * fair[1] - stake * fair[0]

    dkfair = fair_odds(dk[0], dk[1])

    if a_ev > b_ev:
        bet = bankroll * kelly_criterion(1, dkfair[0], dkfair[1])
        print(f'odds: {dk[0]}')
        print(f'width: {width(pin[0], pin[1])}')
        print(f'ev: {a_ev}')
        print(f'optimal_bet: ${round(bet, 2)}')
    else:
        bet = bankroll * kelly_criterion(1, dkfair[0], dkfair[1])
        print(f'odds: {dk[1]}')
        print(f'width: {width(pin[0], pin[1])}')
        print(f'ev: {b_ev}')
        print(f'optimal_bet: ${round(bet, 2)}')

    # return ((dk[0], a_ev), (dk[1], b_ev), bet)
    return None